## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [6]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
import os

file_path = "input.txt"
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

if not os.path.exists(file_path):
    !wget {url}
else:
    print(f"'{file_path}' already exists, skipping download.")


'input.txt' already exists, skipping download.


In [1]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# here are all the unique characters that occur in this text
# 本文中出现的所有独特字符
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

In [6]:
# For the mapping from string to integer
print("Mapping from characters to integers:")
for ch, i in stoi.items():
    print(f"'{ch}': {i}")

# Add some spacing
print("\n" + "-"*50 + "\n")

# For the mapping from integer to string
print("Mapping from integers to characters:")
for i, ch in itos.items():
    print(f"{i}: '{ch}'")


Mapping from characters to integers:
'
': 0
' ': 1
'!': 2
'$': 3
'&': 4
''': 5
',': 6
'-': 7
'.': 8
'3': 9
':': 10
';': 11
'?': 12
'A': 13
'B': 14
'C': 15
'D': 16
'E': 17
'F': 18
'G': 19
'H': 20
'I': 21
'J': 22
'K': 23
'L': 24
'M': 25
'N': 26
'O': 27
'P': 28
'Q': 29
'R': 30
'S': 31
'T': 32
'U': 33
'V': 34
'W': 35
'X': 36
'Y': 37
'Z': 38
'a': 39
'b': 40
'c': 41
'd': 42
'e': 43
'f': 44
'g': 45
'h': 46
'i': 47
'j': 48
'k': 49
'l': 50
'm': 51
'n': 52
'o': 53
'p': 54
'q': 55
'r': 56
's': 57
't': 58
'u': 59
'v': 60
'w': 61
'x': 62
'y': 63
'z': 64

--------------------------------------------------

Mapping from integers to characters:
0: '
'
1: ' '
2: '!'
3: '$'
4: '&'
5: '''
6: ','
7: '-'
8: '.'
9: '3'
10: ':'
11: ';'
12: '?'
13: 'A'
14: 'B'
15: 'C'
16: 'D'
17: 'E'
18: 'F'
19: 'G'
20: 'H'
21: 'I'
22: 'J'
23: 'K'
24: 'L'
25: 'M'
26: 'N'
27: 'O'
28: 'P'
29: 'Q'
30: 'R'
31: 'S'
32: 'T'
33: 'U'
34: 'V'
35: 'W'
36: 'X'
37: 'Y'
38: 'Z'
39: 'a'
40: 'b'
41: 'c'
42: 'd'
43: 'e'
44: 'f'
45: 'g'
46: '

In [8]:
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [9]:
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      